Notebook to look at encoding models of A00c Activity; here we consider all A00c cells together (without regard for where manipulations were delivered or cell identity)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import pickle

from janelia_core.stats.regression import grouped_linear_regression_boot_strap
from janelia_core.stats.regression import visualize_boot_strap_results

from keller_zlatic_vnc.linear_modeling import one_hot_from_table
from keller_zlatic_vnc.linear_modeling import color_grp_vars

In [3]:
%matplotlib qt

In [4]:
# Helps make sure our plots will have the same size
plt.rcParams['figure.constrained_layout.use'] = True

## Parameters go here

In [7]:
data_dir = r'\\dm11\bishoplab\projects\keller_vnc\data\extracted_dff'
data_file = 'A00c_activity.pkl'

# Specify variables that we predict from
beh_before = ['Q', 'F', 'B']
beh_after = ['Q', 'F', 'B']
enc_beh_interactions = True
enc_subjects = True
closure = True # True if the only events we consider must start with a before_beh 
               # behavior and end with an beh_after behavior

# How many boot strap samples we use in each analysis
n_bs_smps = 1000

# Determine how we will display results
font_size = 15
alpha = .01

# Determine what type of manipulation events we look at
manip_type = 'A9' # 'both', 'A4' or 'A9'

# Determine which variables we are going to display results for
show_subj_coefs = True

## Load the data

In [8]:
data_path = Path(data_dir) / data_file
with open(data_path, 'rb') as f:
    data = pickle.load(f)
data = pd.DataFrame(data)

In [10]:
data

,subject_id,cell_id,event_id,beh_before,beh_after,dff_before,dff_during,dff_after,man_tgt
1121,1102L3CL,2.0,0,B,B,0.022125,0.006751,-0.001554,A9
1122,1102L3CL,2.0,1,F,T,0.001597,0.112738,0.273001,A9
1123,1102L3CL,2.0,2,F,F,0.044350,0.033995,0.028438,A9
1124,1102L3CL,2.0,3,F,T,0.001753,0.029739,0.032805,A9
1125,1102L3CL,3.0,0,B,B,0.028357,0.043027,0.015061,A9
...,...,...,...,...,...,...,...,...,...
1537,1108L3,6.0,2,Q,F,0.016744,0.791211,0.939961,A9
1538,1108L3,6.0,3,Q,F,0.011291,0.725406,0.856928,A9
1539,1108L3,6.0,4,Q,F,0.016901,0.705781,0.782500,A9
1540,1108L3,6.0,5,Q,P,0.009434,0.648214,0.664910,A9


## Down select to only the manipulation events we want to consider

In [9]:
if manip_type == 'A4':
    print('Analyzing only A4 manipulation events.')
    data = data[data['man_tgt'] == 'A4']
elif manip_type == 'A9':
    print('Analyzing only A9 manipulation events.')
    data = data[data['man_tgt'] == 'A9']
else:
    print('Analyzing all manipulation events.')

Analyzing only A9 manipulation events.


## Enforce closure if needed

In [8]:
if closure:
    print('Enforcing closure.')
    before_closure = np.asarray([b in set(beh_before) for b in data['beh_before']], 
                                dtype=bool)
    after_closure = np.asarray([b in set(beh_after) for b in data['beh_after']], 
                                dtype=bool)
    closure = np.logical_and(before_closure, after_closure)
    
    data = data[closure]

Enforcing closure.


## Get rid of rows of data that have no behavior of interest

In [9]:
before_ignore = np.asarray([b not in set(beh_before) for b in data['beh_before']], 
                                dtype=bool)
after_ignore = np.asarray([b not in set(beh_after) for b in data['beh_after']], 
                                dtype=bool)

ignore_rows = np.logical_and(before_ignore, after_ignore)

data = data[np.logical_not(ignore_rows)]

## Get groups of data (a group corresponds to each subject)

In [10]:
unique_ids = data['subject_id'].unique()
g = np.zeros(len(data))
for u_i, u_id in enumerate(unique_ids):
    g[data['subject_id'] == u_id] = u_i

## Pull out $\Delta F / F$

In [11]:
dff_before = data['dff_before'].to_numpy()
dff_after = data['dff_after'].to_numpy()

## Now we fit linear models with user specified options

In [12]:
one_hot_data, one_hot_vars = one_hot_from_table(data, 
                                                beh_before=beh_before, 
                                                beh_after=beh_after,
                                                enc_subjects=enc_subjects, 
                                                enc_beh_interactions=enc_beh_interactions)
if not enc_subjects:
    one_hot_vars.append('mean')

In [ ]:
pre_reg_rs = grouped_linear_regression_boot_strap(x=one_hot_data, 
                                                  y=dff_before, 
                                                  g=g, 
                                                  n_bs_smps=n_bs_smps, 
                                                  include_mean=(not enc_subjects))

In [ ]:
post_reg_rs = grouped_linear_regression_boot_strap(x=one_hot_data, 
                                                  y=dff_after, 
                                                  g=g, 
                                                  n_bs_smps=n_bs_smps, 
                                                  include_mean=(not enc_subjects))

### Wrapper function for plotting

In [ ]:
def display_results(rs, var_strs, ylim, title_str):
    
    sig_y = .05*(ylim[1] - ylim[0]) + ylim[0]
    
    ax = visualize_boot_strap_results(bs_values=rs[0],
                                      var_strs=var_strs, 
                                      theta = rs[1],
                                      var_clrs=color_grp_vars(var_strs),
                                      violin_plots=True,
                                      alpha=alpha, 
                                      sig_y_vl=sig_y)
 
    ph = plt.ylabel('Coef Vls', fontsize=font_size)
    ax.tick_params(axis="x", labelsize=font_size)
    ax.tick_params(axis="y", labelsize=font_size)
    ax.set_ylim(ylim)

    plt.title(title_str, fontsize=font_size, pad=50)
    fig = plt.gcf()
    fig.set_size_inches(12, 8)

## Figure out how many subjects went into the analysis

In [ ]:
n_subjects = len(np.unique(g))
print('Number of subjects in analysis: ' + str(n_subjects))

### Figure out how we are going to end our titles

In [ ]:
if manip_type == 'both':
    title_end_str = ', both A4 & A9'
else:
    title_end_str = ', ' + manip_type

### Figure out which vars we are going to show

In [ ]:
if not show_subj_coefs and enc_subjects:
    last_ind = len(one_hot_vars) - n_subjects
    var_inds = slice(0, last_ind)
else:
    var_inds = slice(0, len(one_hot_vars))

In [ ]:
plot_pre_reg_rs = copy.deepcopy(pre_reg_rs)
plot_pre_reg_rs[0] = plot_pre_reg_rs[0][:, var_inds]

plot_post_reg_rs = copy.deepcopy(post_reg_rs)
plot_post_reg_rs[0] = plot_post_reg_rs[0][:, var_inds]

plot_one_hot_vars = one_hot_vars[var_inds]

## Display pre-manipulation $\Delta F/F$ Results

In [ ]:
title_str = 'Encoding for Pre Manipulation $\Delta F /F$ Activity' + title_end_str
display_results(rs=plot_pre_reg_rs, var_strs=plot_one_hot_vars, ylim=[-.05, .05], 
                title_str=title_str)

## Display post-manipulation $\Delta F/F$ Results

In [ ]:
title_str = 'Encoding for Post Manipulation $\Delta F /F$ Activity' + title_end_str
display_results(rs=plot_post_reg_rs, var_strs=plot_one_hot_vars, ylim=[-.5, .5], 
                title_str=title_str)